# MTA_EDA_Project1 

In [1]:
import pandas as pd #import pandas  as pd for sort data 
import datetime  #for sort data by date and time
import matplotlib.pyplot as plt #to plot data 
import numpy as np
from sqlalchemy import create_engine
import seaborn as sns #to plot 
%matplotlib inline 

### if you want to run code remove hash below
* the cell below is for download the date set from MTA website

In [ ]:
def Data(week_numbers):
    URL = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    Data = []
    for week_number in week_numbers:
        file = URL.format(week_number)
        Data.append(pd.read_csv(file))
    return pd.concat(Data)
 
week_numbers = [210501,210508,210515,210522,210529,210605,210612,210619,210626,210703
                ,210710,210717,210724,210731,210807]
Data_DB = Data(week_numbers)

Create database in sqlite and upload file to sql and read it from sql again as dataframe

In [ ]:
engine = create_engine("sqlite:///DB_MTA.db") #to create database

* below cell is to write dateset into database 

In [ ]:
#Data_DB.to_sql("DATA",engine,if_exists="append",index=False) 

In [ ]:
Data_df= pd.read_sql("SELECT * FROM DATA;",engine) #to read from database to dataframe
Data_df

#### Data cleaning 
* cleaning data and remove space in columns 
* drop duplicates 


In [ ]:
Data_df.columns = [column.strip() for column in Data_df.columns] # strip space in columns 
Data_df.columns

In [ ]:
(Data_df.groupby(["C/A","UNIT","SCP","STATION","DIVISION","DATE","TIME"]).ENTRIES.count().reset_index()
 .sort_values("ENTRIES",ascending=False))
#to check duplicate rows

In [ ]:
Data_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE","TIME"],inplace=True, ascending=False)
Data_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE","TIME"], inplace=True)
Data_df
#drop_duplicates

In [ ]:
Data_df["DATE_TIME"]= pd.to_datetime(Data_df.DATE+' '+Data_df.TIME) #new column for date and time

In [ ]:
Data_df["WEEKDAY"]=Data_df["DATE_TIME"].dt.weekday
Data_df
#add columns weekday 

### sort data 
* Groupby data by time to know entry for each station per time 


In [ ]:
Data_time = (Data_df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE","TIME"],
                               as_index=False).ENTRIES.first()) # groupby time 

In [ ]:
Data_time["Daily_ENTRIES_T"]=Data_time.groupby(["C/A", "UNIT", "SCP", "STATION"])[["ENTRIES"]].diff()
#find difference between entries per time 

In [ ]:
Data_time.dropna(subset=["Daily_ENTRIES_T"], axis=0, inplace = True) #drop NaN

In [ ]:
Data_df.TIME=pd.to_datetime(Data_df.TIME) #change to datatime 

In [ ]:
Data_time["Daily_fixed"]=(np.where(((Data_time.ENTRIES ==0) & (Data_time.Daily_ENTRIES_T < 0)),0,
                           np.where((Data_time.ENTRIES !=0) & (Data_time.Daily_ENTRIES_T < 0)
                           ,abs(Data_time.Daily_ENTRIES_T),Data_time.Daily_ENTRIES_T)))
Data_time.reset_index()
# find values =0 for ENTRIES and Daily_ENTRIES_T < 0 put it 0 and for ENTRIES !=0 and Daily_ENTRIES_T < 0 
# replace it with abs values

* Groupby data by Date to know entry for each station per day

In [ ]:
Data_Day= (Data_df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE","WEEKDAY"],
                           as_index=False).ENTRIES.first()) #same as above but per day 

In [ ]:
Data_Day["Daily_ENTRIES_D"]=Data_Day.groupby(["C/A", "UNIT", "SCP", "STATION"])[["ENTRIES"]].diff()

In [ ]:
Data_Day.dropna(subset=["Daily_ENTRIES_D"], axis=0, inplace = True)

In [ ]:
Data_Day["Daily_fixed"]=(np.where(((Data_Day.ENTRIES ==0) & (Data_Day.Daily_ENTRIES_D < 0)),0,
                          np.where((Data_Day.ENTRIES !=0) & (Data_Day.Daily_ENTRIES_D < 0)
                          ,abs(Data_Day.Daily_ENTRIES_D),Data_Day.Daily_ENTRIES_D)))
Data_Day

* check missing values in data by choose randomly for some station

In [ ]:
mask = (
        (Data_time["C/A"] == "A050") &
        (Data_time["UNIT"] == "R088") & 
        (Data_time["SCP"]== "00-05-01")&
        (Data_time["STATION"] == "CORTLANDT ST")&
        (Data_time["DATE"]=="06/15/2021"))
missing_v1=Data_time[mask]

missing_v1.reset_index()

#to draw missing 

In [ ]:
mask = (
        (Data_time["C/A"] == "A050") &
        (Data_time["UNIT"] == "R088") & 
        (Data_time["SCP"]== "00-05-01")&
        (Data_time["STATION"] == "CORTLANDT ST")&
        (Data_time["DATE"]=="06/16/2021"))
missing_v2=Data_time[mask]
missing_v2.reset_index()

* Check values not equale 0 and bigeer than 0 

In [ ]:
df=Data_Day[((Data_Day.ENTRIES !=0) & (Data_Day.Daily_ENTRIES_D > 0))]
df

* I assumed that the data with more than 5000k entries is correct because if we put it 10k the difference will be 3 rows

In [ ]:
Data_5=(df[df.Daily_fixed<5000])

Data_5["DATE_DT"]= pd.to_datetime(Data_5["DATE"])
Data_5.sort_values("Daily_fixed",ascending=False)

In [ ]:
Data_5["WEEK"]=Data_5.DATE_DT.dt.week
Data_5.reset_index()

In [ ]:
Data_weekly=Data_5.groupby(["STATION","WEEK"]).Daily_fixed.sum().reset_index().sort_values("Daily_fixed",ascending=False)
Data_weekly=Data_weekly.head(14).reset_index()
Data_weekly.sort_values("WEEK",ascending=True)

In [ ]:
plt.figure(figsize = [9,6])
sns.set_style("white")
plt.grid

plt.xticks([18,19,20,21,22,23,24,25,26,27,28,29,30,31],["1","2","3","4","5","6","7","8","9","10","11","12","13","14"])
plt.xlabel("WEEKS",fontsize = 10, weight = 'bold')
plt.ylabel('ENTRY STATION PER WEEK',fontsize = 10, weight = 'bold')
plt.title("ENTRY FOR STATION (34 ST-PENN STA) PER WEEK",fontsize = 15, weight = 'bold')
plt.bar(Data_weekly['WEEK'],Data_weekly["Daily_fixed"],width = 0.7,color=("maroon"),edgecolor='black');




In [ ]:
plt.figure(figsize = [14,4])
plt.suptitle('BEHAVIOR OF CORTLANDT ST STATION',fontsize = 12,weight = 'bold',c="maroon")
plt.subplot(1,2,1)
plt.plot(missing_v1["TIME"],(missing_v1["ENTRIES"]),"ko",linestyle="-",c="maroon")
plt.grid()
plt.xticks(rotation=40)
plt.xticks(["01:00:00","05:00:00","09:00:00",
          "21:00:00"],
           ["01:00","05:00","09:00","21:00"])
plt.title('15/06/2021',fontsize=10);

plt.subplot(1,2,2)
plt.plot(missing_v2["TIME"],(missing_v2["ENTRIES"]),"ko",linestyle="-",c="maroon")
plt.xticks(["01:00:00","05:00:00","09:00:00",
            "13:00:00","17:00:00","21:00:00"],
           ["01:00","05:00","09:00","13:00","17:00","21:00"])
#plt.xticks(["1900-01-01 01:00:00","1900-01-01 05:00:00","1900-01-01 09:00:00",
#             "1900-01-01 13:00:00","1900-01-01 17:00:00","1900-01-01 21:00:00"],
#            ["01:00","05:00","09:00","13:00","17:00","21:00"])


plt.title('16/06/2021',fontsize=10);


plt.xticks(rotation=40)

plt.grid()

sns.set_style("white")


In [ ]:
top15=Data_5.groupby("STATION").Daily_fixed.sum().reset_index().sort_values("Daily_fixed",ascending=False).head(15)
top15

In [ ]:
plt.figure(figsize = [14,8])

sns.barplot(x='Daily_fixed',y = 'STATION',data=top15,palette="rocket")
sns.despine()
plt.xlabel("TOTAL ENTERY PER 14-WEEKS",fontsize = 10, weight = 'bold')
plt.ylabel('TOP ENTRY STATION',fontsize = 10, weight = 'bold')
plt.xticks([0,1000000,2000000,3000000,4000000],["0","1M","2M","3M","4M"])
plt.title("MOST ENTRY STATIONS",fontsize = 20, weight = 'bold')
sns.despine()
sns.set_style("white")

In [ ]:
Data_Day["DATE_DT"] = pd.to_datetime(Data_Day["DATE"])
Data_Day["WEEK"]=Data_Day.DATE_DT.dt.week
Data_Day

In [ ]:
mask = ((Data_Day["C/A"] == "N067") &
        (Data_Day["UNIT"] == "R012") &
        
        (Data_Day["STATION"] == "34 ST-PENN STA")
        )

ST_34=Data_Day[mask]
ST_34.sort_values("Daily_fixed",ascending=False)
tt=ST_34.reset_index()
# 34 ST-PENN STA
tt

In [ ]:
mask = ((ST_34["C/A"] == "N067") &
        (ST_34["UNIT"] == "R012") & 
        (ST_34["STATION"] == "34 ST-PENN STA")&
         (ST_34["WEEK"] == 29)
      )
#len(Data_df[mask])
ST_34[mask]

In [ ]:
ST_two_week=ST_34.groupby(["DATE_DT","STATION",])[["Daily_fixed"]].sum()
ST_two_week.reset_index(inplace=True)
ST_two_week["WEEK"]=ST_two_week.DATE_DT.dt.week
ST_two_week
ST_two_week.reset_index(inplace=True)


In [ ]:
mask = (
        (ST_two_week["STATION"] == "34 ST-PENN STA")&
         (ST_two_week["WEEK"] >= 29)&
    (ST_two_week["WEEK"] <= 30)
    
      )
#len(Data_df[mask])
ST_two_week1=ST_two_week[mask]

In [ ]:
plt.figure(figsize = [16,5])
#plt.xticks([0,1,2,3,4,5,6],["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"])
plt.plot(ST_two_week.DATE_DT,ST_two_week.Daily_fixed, color=("#850707"),linestyle="-")
#plt.yticks([80000,90000,100000,110000,120000])
plt.grid()
plt.legend(["TOTAL ENTRY FOR 3 MONTHS "],shadow = True, loc = 2);
plt.xlabel("DATE",fontsize = 15, weight = 'bold')
plt.ylabel('TOTAL ENTRY PER DAY',fontsize = 15, weight = 'bold')
plt.title("ENTRY FOR 3 MONTHS (34 ST-PENN STA) STATION",fontsize = 20, weight = 'bold')

In [ ]:
ST_34perweek=ST_34.groupby(["WEEKDAY","STATION"])[["Daily_fixed"]].sum()
ST_34perweek.reset_index(inplace=True)
ST_34perweek

In [ ]:
plt.figure(figsize = [14,4])


plt.plot(ST_two_week1.DATE_DT,ST_two_week1.Daily_fixed,"ko", color=("#850707"),linestyle="-")
#plt.yticks([80000,90000,100000,110000,120000])
plt.grid()
plt.legend(['TOTAL ENTRY FOR 3 MONTHS'],shadow = True, loc = 3);
plt.xlabel("DATE",fontsize = 15, weight = 'bold')
plt.ylabel('TOTAL ENTRY',fontsize = 15, weight = 'bold')
plt.title("ENTRY FOR STATION (34 ST-PENN STA)",fontsize = 20, weight = 'bold')
plt.xticks(rotation=40)

In [ ]:
ST_34["WEEKDAY"]=Data_df["DATE_TIME"].dt.weekday

In [ ]:
mask = (
        (Data_df["STATION"] == "GRD CNTRL-42 ST")&
        (Data_df["DATE"]=="05/07/2021"))
#len(Data_df[mask])
Data_df[mask].tail(5)

In [ ]:
mask = ((Data_Day["C/A"] == "A015") &
        (Data_Day["UNIT"] == "R081") & 
        (Data_Day["STATION"] == "49 ST")&
        (Data_Day["SCP"] == "00-00-02")&
       (Data_Day["DATE_DT"].dt.month == 5)&
       (Data_Day["DATE_DT"].dt.day >=14)&
        (Data_Day["DATE_DT"].dt.day <=20))
test=Data_Day[mask]
test.reset_index()

In [ ]:
test.sort_index()

In [ ]:
test.barh(ss.WEEKDAY,ss.Daily_fixed);